In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, SGD
from torchsummary import summary
from torch.cuda.amp import autocast, GradScaler

import numpy as np
import gzip
import pickle
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, cohen_kappa_score, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

from tqdm import tqdm
import gc
import time
import random



import sys
sys.path.append('..')
# from slp_package.slp_functions import create_merged_game_data_df
from slp_package.input_dataset import InputDataSet
import slp_package.pytorch_functions as slp_pytorch_functions

def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using CUDA
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)
torch.cuda.is_available()

In [ ]:
def asses_model(model_name, y_pred, y_test, labels_order):
    print()
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    kappa = cohen_kappa_score(y_pred, y_test)

    # Print accuracy and Cohen Kappa score with explanations
    print(f'Accuracy of {model_name}: {accuracy:.4f}')
    print(f'Cohen Kappa Score of {model_name}: {kappa:.4f}')

    # Calculate the normalized predicted label count
    unique_pred, counts_pred = np.unique(y_pred, return_counts=True)
    unique_test, counts_test = np.unique(y_test, return_counts=True)
    normalized_counts_pred = {k: v / counts_test[np.where(unique_test == k)[0][0]] for k, v in zip(unique_pred, counts_pred)}
    
    # Calculate the percent the model over or under predicted the labels using the specified label order
    sorted_values = [normalized_counts_pred[k] - 1 if k in normalized_counts_pred else 0 for k in labels_order]

    # Plotting the percent the model over or under predicted the labels
    plt.figure(figsize=(2*len(labels_order), 4))
    plt.bar(labels_order, sorted_values, color=['green' if x > 0 else 'blue' for x in sorted_values])
    plt.title(f'Percent Model {model_name} Over or Under Predicted Labels')
    plt.xlabel('Labels')
    plt.ylabel('Percent Over/Under Prediction')
    
    # Center y-axis and set equal extension above and below
    max_extent = max(abs(min(sorted_values)), abs(max(sorted_values))) * 1.05
    plt.ylim(-max_extent, max_extent)
    plt.axhline(y=0, color='gray', linewidth=0.8)
    plt.show()

    # Display each confusion matrix on its own row
    for norm in [None, 'true', 'pred']:
        plt.figure(figsize=(len(labels_order)+5, len(labels_order)+5))
        ax = plt.gca()
        ConfusionMatrixDisplay.from_predictions(
            y_test, y_pred, normalize=norm, ax=ax,
            xticks_rotation='vertical', labels=labels_order
        )
        ax.title.set_text(f'{model_name} Confusion Matrix ({"Not Normalized" if norm is None else "Normalized by " + norm})')
        plt.tight_layout()
        plt.show()
        

# def asses_model(model_name, y_pred, y_test, labels_order):
#     # Calculate metrics
#     accuracy = accuracy_score(y_test, y_pred)
#     kappa = cohen_kappa_score(y_pred, y_test)

#     # Print accuracy and Cohen Kappa score
#     print(f'Accuracy of {model_name}: {accuracy:.4f}')
#     print(f'Cohen Kappa Score of {model_name}: {kappa:.4f}')

#     # Calculate the normalized predicted label count
#     unique_pred, counts_pred = np.unique(y_pred, return_counts=True)
#     unique_test, counts_test = np.unique(y_test, return_counts=True)
#     normalized_counts_pred = {k: v / counts_test[np.where(unique_test == k)[0][0]] for k, v in zip(unique_pred, counts_pred) if k in unique_test}
    
#     # Calculate the percent the model over or under predicted the labels
#     sorted_values = [normalized_counts_pred.get(k, 0) - 1 for k in labels_order]

#     # Plotting the percent the model over or under predicted the labels
#     plt.figure(figsize=(2*len(labels_order), 4))
#     plt.bar(labels_order, sorted_values, color=['green' if x > 0 else 'blue' for x in sorted_values])
#     plt.title(f'Percent {model_name} Over or Under Predicted Labels')
#     plt.xlabel('Labels')
#     plt.ylabel('Percent Over/Under Prediction')

#     # Center y-axis and set equal extension above and below
#     max_extent = max(abs(min(sorted_values)), abs(max(sorted_values))) * 1.05
#     plt.ylim(-max_extent, max_extent)
#     plt.axhline(y=0, color='gray', linewidth=0.8)
#     plt.show()

#     # Display each confusion matrix on its own row
#     for norm in [None, 'true', 'pred']:
#         plt.figure(figsize=(len(labels_order)+5, len(labels_order)+5))
#         ax = plt.gca()
#         ConfusionMatrixDisplay.from_predictions(
#             y_test, y_pred, normalize=norm, ax=ax,
#             display_labels=labels_order,
#             xticks_rotation='vertical'
#         )
#         ax.title.set_text(f'{model_name} Confusion Matrix ({"Not Normalized" if norm is None else "Normalized by " + norm})')
#         ax.set_xlabel('Predicted labels')
#         ax.set_ylabel('True labels')
#         ax.set_xticklabels(labels_order)  # Explicitly setting x-axis labels
#         ax.set_yticklabels(labels_order[::-1])  # Setting y-axis labels to reverse order
#         plt.tight_layout()
#         plt.show()

# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.metrics import accuracy_score, cohen_kappa_score, ConfusionMatrixDisplay

# def asses_model(model_name, y_pred, y_test, labels_order, label_decoder):
#     """
#     Evaluate and visualize the model's performance with decoded labels.

#     Args:
#         model_name (str): Name of the model for display purposes.
#         y_pred (np.ndarray): Encoded predictions from the model.
#         y_test (np.ndarray): Encoded true labels.
#         labels_order (list): Ordered list of labels for display.
#         label_decoder (dict): Dictionary to decode labels from encoded labels to string labels.
#     """
#     # Decode predictions and true labels
#     decoded_y_pred = [label_decoder.get(label, "Unknown") for label in y_pred]
#     decoded_y_test = [label_decoder.get(label, "Unknown") for label in y_test]

#     # Calculate metrics
#     accuracy = accuracy_score(decoded_y_test, decoded_y_pred)
#     kappa = cohen_kappa_score(decoded_y_pred, decoded_y_test)
#     print(f'Accuracy of {model_name}: {accuracy:.4f}')
#     print(f'Cohen Kappa Score of {model_name}: {kappa:.4f}')

#     # Normalize the predicted label counts
#     unique_pred, counts_pred = np.unique(decoded_y_pred, return_counts=True)
#     unique_test, counts_test = np.unique(decoded_y_test, return_counts=True)
#     normalized_counts_pred = {k: v / counts_test[np.where(unique_test == k)[0][0]] for k, v in zip(unique_pred, counts_pred) if k in unique_test}
    
#     # Calculate and plot the percent the model over or under predicted the labels
#     sorted_values = [normalized_counts_pred.get(k, 0) - 1 for k in labels_order]
#     plt.figure(figsize=(2*len(labels_order), 4))
#     plt.bar(labels_order, sorted_values, color=['green' if x > 0 else 'blue' for x in sorted_values])
#     plt.title(f'Percent {model_name} Over or Under Predicted Labels')
#     plt.xlabel('Labels')
#     plt.ylabel('Percent Over/Under Prediction')
#     max_extent = max(abs(min(sorted_values)), abs(max(sorted_values))) * 1.05
#     plt.ylim(-max_extent, max_extent)
#     plt.axhline(y=0, color='gray', linewidth=0.8)
#     plt.show()

#     # Display each confusion matrix
#     for norm in [None, 'true', 'pred']:
#         plt.figure(figsize=(len(labels_order)+5, len(labels_order)+5))
#         ax = plt.gca()
#         ConfusionMatrixDisplay.from_predictions(
#             decoded_y_test, decoded_y_pred, normalize=norm, ax=ax,
#             display_labels=labels_order,
#             xticks_rotation='vertical'
#         )
#         ax.title.set_text(f'{model_name} Confusion Matrix ({"Not Normalized" if norm is None else "Normalized by " + norm})')
#         ax.set_xlabel('Predicted labels')
#         ax.set_ylabel('True labels')
#         ax.set_xticklabels(labels_order)
#         ax.set_yticklabels(labels_order[::-1])
#         plt.tight_layout()
#         plt.show()


In [ ]:
source_data = ['ranked','public','mango']

general_features = {
    'stage_name': ['FOUNTAIN_OF_DREAMS','FINAL_DESTINATION','BATTLEFIELD','YOSHIS_STORY','POKEMON_STADIUM','DREAMLAND'],
    'num_players': [2],
    'conclusive': [True]
}
player_features = {
    # 'netplay_code': ['MANG#0'],
    # 'character_name': ['FALCO'],
    # 'character_name': ['FOX', 'FALCO', 'MARTH', 'CAPTAIN_FALCON', 'SHEIK'],
    # 'character_name': ['FOX', 'CAPTAIN_FALCON', 'SHEIK', 'FALCO', 'GAME_AND_WATCH', 'MARTH', 'LINK', 'ICE_CLIMBERS', 'SAMUS', 'GANONDORF', 'BOWSER', 'MEWTWO', 'YOSHI', 'PIKACHU', 'JIGGLYPUFF', 'NESS', 'DR_MARIO', 'MARIO', 'PEACH', 'ROY', 'LUIGI', 'YOUNG_LINK', 'DONKEY_KONG', 'PICHU', 'KIRBY'],
    'character_name': ['FOX', 'CAPTAIN_FALCON', 'SHEIK', 'FALCO', 'GAME_AND_WATCH', 'MARTH', 'LINK', 'ICE_CLIMBERS', 'SAMUS', 'GANONDORF', 'BOWSER', 'MEWTWO', 'YOSHI', 'PIKACHU', 'JIGGLYPUFF', 'NESS', 'DR_MARIO', 'PEACH', 'LUIGI', 'DONKEY_KONG'],
    'type_name': ['HUMAN']
    
}
opposing_player_features = {
    # 'character_name': ['MARTH'],
    # 'netplay_code': ['KOD#0', 'ZAIN#0']
    'type_name': ['HUMAN']
}
label_info = {
    'source': ['player'], # Can be 'general', 'player
    # 'feature': ['netplay_code']
    'feature': ['character_name']
}

In [ ]:
# We classify 5 characters on competitive stages

source_data = ['ranked']

general_features = {
    'stage_name': ['FOUNTAIN_OF_DREAMS','FINAL_DESTINATION','BATTLEFIELD','YOSHIS_STORY','POKEMON_STADIUM','DREAMLAND'],
    'num_players': [2],
    'conclusive': [True]
}
player_features = {
    # 'netplay_code': ['MANG#0'],
    'character_name': ['FOX'],
    'type_name': ['HUMAN']
    
}
opposing_player_features = {
    'character_name': ['FOX', 'FALCO', 'MARTH', 'CAPTAIN_FALCON', 'SHEIK'],
    # 'netplay_code': ['KOD#0', 'ZAIN#0'],
    'type_name': ['HUMAN']
}
label_info = {
    'source': ['opposing_player'], # Can be 'general', 'player', 'opposing_player'
    # 'feature': ['netplay_code']
    'feature': ['character_name']
}
    

In [ ]:
dataset = InputDataSet(source_data, general_features, player_features, opposing_player_features, label_info)

print(dataset.dataset['labels'].value_counts())

In [ ]:
labels_order =  dataset.number_of_segments_per_game(12,100000)
print(labels_order)
labels_order = labels_order['Label'].values


In [ ]:
train_df, test_df  = dataset.train_test_split_dataframes(test_ratio = .20, val = False)

In [ ]:
train_df.head()

In [ ]:
labels_unique = train_df['labels'].unique()
encoded_labels_unique = train_df['encoded_labels'].unique()
label_decoder = zip(labels_unique, encoded_labels_unique)
label_decoder = dict(zip(encoded_labels_unique, labels_unique)) 
print(label_decoder)

In [ ]:
# from ResNet_Model import ResNet50, ResNet101
# from ResNet_Model_Relative import ResNet50
from ResNet_Model_Relative_2 import ResNet50

model = ResNet50(num_classes=20, channels=9).to('cuda')


In [ ]:
# # Create a dummy input with the correct shape
# dummy_input = torch.randn(1, 9, 4096)  # batch size of 1, 9 channels, 4096 length
# try:
#     with torch.no_grad():
#         model(dummy_input.to(model.device))  # Ensure the dummy input is on the same device as the model
# except Exception as e:
#     print("Error during test forward pass: ", str(e))



In [ ]:
summary(model, input_size=(9, 2**12))

In [ ]:
# model = torch.compile(model, mode = 'default').to('cuda')
model = torch.compile(model, mode='max-autotune')
# model = model.to('cuda')

In [ ]:
loaders = slp_pytorch_functions.prepare_data_loaders_no_val(train_df, test_df, 32*2, 16)
criterion = nn.CrossEntropyLoss(reduction = 'sum')
optimizer = Adam(model.parameters(), lr=0.001)
# optimizer = SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)
slp_pytorch_functions.train_model(model, criterion,optimizer, loaders, 'cuda', 2 )
y_pred = slp_pytorch_functions.predict(model, loaders['test'], 'cuda' )
# y_pred = slp_pytorch_functions.predict(model, loaders['train'], 'cuda' )

In [ ]:
asses_model('ResNet-50', np.array([label_decoder.get(item, "Unknown") for item in y_pred]), np.array(test_df['labels']), labels_order)


In [ ]:
# x = [3,4,6,3]
# print(1 + 2 * (x[0] * 4 + x[1] * 8 + x[2] * 16 + x[3] * 32))
# print('Memory of ResNet50:', (1 + 2 * (x[0] * 4 + x[1] * 8 + x[2] * 16 + x[3] * 32)) / 60, 'seconds')
# x = [3,4,23,3]
# print(1 + 2 * (x[0] * 4 + x[1] * 8 + x[2] * 16 + x[3] * 32))
# print('Memory of ResNet101:', (1 + 2 * (x[0] * 4 + x[1] * 8 + x[2] * 16 + x[3] * 32)) / 60, 'seconds')
# x = [3,8,36,3]
# print(1 + 2 * (x[0] * 4 + x[1] * 8 + x[2] * 16 + x[3] * 32))
# print('Memory of ResNet152:', (1 + 2 * (x[0] * 4 + x[1] * 8 + x[2] * 16 + x[3] * 32)) / 60, 'seconds')


In [ ]:
# x = [3, 4, 4]
# plus = 3 + 2
# multiplier = 4
# for val in x:
#     plus += multiplier * val
#     multiplier *= 2
# print(1 + 2 * plus)
    
# print(1 + 2 * (x[0] * 4 + x[1] * 8 + x[2] * 16 + x[3] * 32))

In [ ]:
slp_pytorch_functions.train_model(model, criterion,optimizer, loaders, 'cuda', 2 )
y_pred = slp_pytorch_functions.predict(model, loaders['test'], 'cuda' )
asses_model('ResNet-50', np.array([label_decoder.get(item, "Unknown") for item in y_pred]), np.array(test_df['labels']), labels_order)


In [ ]:
slp_pytorch_functions.train_model(model, criterion,optimizer, loaders, 'cuda', 2 )
y_pred = slp_pytorch_functions.predict(model, loaders['test'], 'cuda' )
asses_model('ResNet-50', np.array([label_decoder.get(item, "Unknown") for item in y_pred]), np.array(test_df['labels']), labels_order)

In [ ]:
slp_pytorch_functions.train_model(model, criterion,optimizer, loaders, 'cuda', 2 )
y_pred = slp_pytorch_functions.predict(model, loaders['test'], 'cuda' )
asses_model('ResNet-50', np.array([label_decoder.get(item, "Unknown") for item in y_pred]), np.array(test_df['labels']), labels_order)